### DE between large clone TRM and small clone TRM

In [ ]:
def DE_to_df(_adata, rank_key, _padj_thresh=0.05, _logfc_thresh=1):
    def process_genes(i, direction):
        _log2foldmask = (_adata.uns[rank_key]['logfoldchanges'][i].astype('double') >= _logfc_thresh) if direction == "up" else (_adata.uns[rank_key]['logfoldchanges'][i].astype('double') <= -_logfc_thresh)
        _pvalmask = _adata.uns[rank_key]['pvals_adj'][i].astype('double') <= _padj_thresh
        _additional = pd.DataFrame({
            i: _adata.uns[rank_key]['names'][i].astype('str')[_log2foldmask & _pvalmask],
            'logfoldchanges_'+i: np.abs(_adata.uns[rank_key]['logfoldchanges'][i].astype('double')[_log2foldmask & _pvalmask])
        })
        _ribo_gene_mask = [gene.startswith('RPL') or gene.startswith('RPS') for gene in _additional[i]]
        _mt_gene_mask = [gene.startswith('MT-') for gene in _additional[i]]
        if len(_ribo_gene_mask)>0 and len(_mt_gene_mask)>0:
            _drop_id = np.array(np.array(_ribo_gene_mask) | np.array(_mt_gene_mask))
            _additional = _additional.sort_values(by='logfoldchanges_'+i, ascending=False)
            _additional = _additional.iloc[_drop_id==False, :].reset_index(drop=True)
        return _additional

    _pass_genes_up = pd.DataFrame()
    _pass_genes_down = pd.DataFrame()

    for i in set(_adata.uns[rank_key]['pvals_adj'].dtype.names):

        _pass_genes_up = pd.concat([_pass_genes_up, process_genes(i, "up")[i]], ignore_index=False, axis=1)
        _pass_genes_down = pd.concat([_pass_genes_down, process_genes(i, "down")[i]], ignore_index=False, axis=1)

    return _pass_genes_up, _pass_genes_down

In [ ]:
def get_df_from_rank(adata, rank_key):
        df = pd.DataFrame(index = adata.uns[rank_key]['names']['True'], columns = ['log2fold','pval_adj'])
        df['log2fold'] = adata.uns[rank_key]['logfoldchanges']['True'].astype('double')
        df['pval_adj'] = adata.uns[rank_key]['pvals_adj']['True'].astype('double')
        return df

In [ ]:
TRM_expansion_DE = dict()
for gtype in ['TCRab CD4','TCRab CD8ab','TCRaa CD8aa']:
    for j in set( adata_all.obs['tissue']):
        for i in range(0,3):
            adata_temp = adatas[i][adatas[i].obs['general type']==gtype,:]
            adata_t = adata_temp[adata_temp.obs['tissue'] == j, :]
            for ct in set(adata_t.obs['general subtype']):
                TRM_expansion_DE[' '.join([adata_t.obs['Donor ID'][0],j,gtype,ct])] = dict()
                
                TRM_cd_t_specific = adata_t[adata_t.obs['general subtype'] == ct,:]      
                TRM_cd_t_specific = TRM_cd_t_specific[TRM_cd_t_specific.obs[j+' top clones'].isin(['1-10','100+']),:]
                TRM_cd_t_specific.obs['top10_or_not']= TRM_cd_t_specific.obs[j+' top clones'] =='1-10'
                TRM_cd_t_specific.obs['top10_or_not']= TRM_cd_t_specific.obs['top10_or_not'].astype(str)
                if TRM_cd_t_specific.obs['top10_or_not'].tolist() == [] or sum(TRM_cd_t_specific.obs['top10_or_not']=='True') <10:
                    print(' '.join([gtype,j,str(i),ct]), 'has no top clone')
                    continue
                else:
                    if 10*sum(TRM_cd_t_specific.obs['top10_or_not']=='True') <sum(TRM_cd_t_specific.obs['top10_or_not']=='False'):
                        print(' '.join([gtype,j,str(i),ct]), 'is not balance')
                        continue
                    else:
                        print(' '.join([gtype,j,str(i),ct]), 'running DE...')
                        print(sum(TRM_cd_t_specific.obs['top10_or_not']=='True'))
                        print(sum(TRM_cd_t_specific.obs['top10_or_not']=='False'))
                        sc.tl.rank_genes_groups(TRM_cd_t_specific, groupby='top10_or_not', key_added='rank_top10_or_not',method = 'wilcoxon')
                        deg_df  = get_df_from_rank(TRM_cd_t_specific, rank_key='rank_top10_or_not')
                        deg_df.to_csv('_'.join([gtype,j,str(i),ct,'_DE_df.csv']).replace('/','_'))
                        pass_genes_up, pass_genes_down = DE_to_df(TRM_cd_t_specific, rank_key='rank_top10_or_not', _padj_thresh = 0.05, _logfc_thresh = 1)
                        print(pass_genes_up)
                        try:
                            if pass_genes_up['True'].tolist() != []:
                                TRM_expansion_DE[' '.join([adata_t.obs['Donor ID'][0],j,gtype,ct])]['1-10'] = pass_genes_up['True'].tolist()
                            if pass_genes_up['False'].tolist() != []:
                                TRM_expansion_DE[' '.join([adata_t.obs['Donor ID'][0],j,gtype,ct])]['100+'] = pass_genes_up['False'].tolist()
                        except:
                            continue

TCRab CD4 PB 0 FOXP3+ Treg is not balance
TCRab CD4 PB 0 Naive/TCM has no top clone
TCRab CD4 PB 0 TCM has no top clone
TCRab CD4 PB 1 FOXP3+ Treg has no top clone
TCRab CD4 PB 1 Naive/TCM has no top clone
TCRab CD4 PB 1 TCM has no top clone
TCRab CD4 PB 2 FOXP3+ Treg is not balance
TCRab CD4 PB 2 Naive/TCM has no top clone
TCRab CD4 PB 2 TCM has no top clone
TCRab CD4 LP 0 Naive/TCM has no top clone
TCRab CD4 LP 0 Tph running DE...
27
192
      True False
0    TRAV4   NaN
1  HLA-DRA   NaN
TCRab CD4 LP 0 FOXP3+ Treg has no top clone
TCRab CD4 LP 0 Mobile TRM has no top clone
TCRab CD4 LP 0 TRM is not balance
TCRab CD4 LP 0 Poised TCM has no top clone
TCRab CD4 LP 1 Naive/TCM has no top clone
TCRab CD4 LP 1 Tph has no top clone
TCRab CD4 LP 1 FOXP3+ Treg has no top clone
TCRab CD4 LP 1 Mobile TRM has no top clone
TCRab CD4 LP 1 TRM is not balance
TCRab CD4 LP 1 Poised TCM has no top clone
TCRab CD4 LP 2 Naive/TCM has no top clone
TCRab CD4 LP 2 Tph has no top clone
TCRab CD4 LP 2 FOXP3+

In [ ]:
import math
def is_nan(value):
    return isinstance(value, float) and math.isnan(value)

TRM_expansion_DE_a = {k: v for k, v in TRM_expansion_DE.items() if v}
for key, subdict in TRM_expansion_DE_a.items():
    for subkey, sublist in subdict.items():
        subdict[subkey] = [item for item in sublist if not is_nan(item)]

data = []

# Loop through the dictionary and flatten it
for cell_type, genes in TRM_expansion_DE.items():
    for category in ['1-10', '100+']:
        for gene in genes.get(category, []):
            data.append([cell_type, gene])

# Convert the list to a DataFrame
df = pd.DataFrame(data, columns=['cell type', 'gene'])
print(df)
#print(df.groupby(['cell type'])['gene'].value_counts())#('gene')['cell type'].value_counts())#apply(set)['GPR183'])
df_drop = df.drop_duplicates(subset = 'gene')
keep_id = [i.startswith('TR')==0 and i.startswith('CD8')==0 for i in df_drop['gene']]
df_drop = df_drop.iloc[keep_id,:]
result_dict = df_drop.groupby('cell type')['gene'].apply(list).to_dict()
print(result_dict)

                            cell type        gene
0       Donor AJKQ118 L TCRab CD4 TRM      TRAV19
1       Donor AJKQ118 L TCRab CD4 TRM    TRAV12-3
2       Donor AJKQ118 L TCRab CD4 TRM       TRBC2
3       Donor AJKQ118 L TCRab CD4 TRM      LGALS1
4       Donor AJKQ118 L TCRab CD4 TRM        NME2
5      Donor AJD3280 LP TCRab CD4 Tph       TRAV4
6      Donor AJD3280 LP TCRab CD4 Tph     HLA-DRA
7   Donor AJD3280 IEL TCRab CD8ab TRM    TRAV13-2
8   Donor AJG2309 IEL TCRab CD8ab TRM     TRBV6-5
9   Donor AJG2309 IEL TCRab CD8ab TRM   TRAV14DV4
10  Donor AJG2309 IEL TCRab CD8ab TRM       TRBC1
11  Donor AJKQ118 IEL TCRab CD8ab TRM      TRBV28
12  Donor AJKQ118 IEL TCRab CD8ab TRM   LINC02446
13  Donor AJKQ118 IEL TCRab CD8ab TRM        CD8A
14  Donor AJKQ118 IEL TCRab CD8ab TRM        CD8B
15  Donor AJKQ118 IEL TCRab CD8ab TRM       SPRY1
16  Donor AJKQ118 IEL TCRab CD8ab TRM      GPR183
17  Donor AJKQ118 IEL TCRab CD8ab TRM       PDE4B
18  Donor AJG2309 PB TCRab CD8ab Teff       TRGV9


In [ ]:
keep_id = [i.startswith('TR')==0 and i.startswith('CD8')==0 for i in df['gene']]
df = df.iloc[keep_id,:]
df

,cell type,gene
3,Donor AJKQ118 L TCRab CD4 TRM,LGALS1
4,Donor AJKQ118 L TCRab CD4 TRM,NME2
6,Donor AJD3280 LP TCRab CD4 Tph,HLA-DRA
12,Donor AJKQ118 IEL TCRab CD8ab TRM,LINC02446
15,Donor AJKQ118 IEL TCRab CD8ab TRM,SPRY1
16,Donor AJKQ118 IEL TCRab CD8ab TRM,GPR183
17,Donor AJKQ118 IEL TCRab CD8ab TRM,PDE4B
19,Donor AJG2309 PB TCRab CD8ab Teff,AC243829.2
20,Donor AJG2309 PB TCRab CD8ab Teff,GZMB
21,Donor AJG2309 PB TCRab CD8ab Teff,IL7R


In [ ]:
result_dict.keys()

dict_keys(['Donor AJD3280 LP TCRab CD4 Tph', 'Donor AJG2309 L TCRab CD8ab TRM', 'Donor AJG2309 PB TCRab CD8ab Teff', 'Donor AJKQ118 IEL TCRab CD8ab TRM', 'Donor AJKQ118 L TCRab CD4 TRM', 'Donor AJKQ118 L TCRab CD8ab TRM', 'Donor AJKQ118 PB TCRab CD8ab Teff'])

In [ ]:
plt.rcParams.update({'font.size': 12, 'font.weight': 'bold','axes.linewidth':2})

for gtype in ['TCRab CD4','TCRab CD8ab']:
    for j in set(adata_all.obs['tissue']):
        for i in range(0,3):
            adata_temp = adatas[i][adatas[i].obs['general type']==gtype,:]
            adata_t = adata_temp[adata_temp.obs['tissue'] == j, :]
            for ct in set(adata_t.obs['general subtype']):
                keyname = ' '.join([adata_t.obs['Donor ID'][0],j,gtype,ct])
                if keyname in result_dict.keys():
                    
                    TRM_cd_t_specific = adata_t[adata_t.obs['general subtype'] == ct,:]      
                    TRM_cd_t_specific = TRM_cd_t_specific[TRM_cd_t_specific.obs[j+' top clones'].isin(['1-10','100+']),:]
                    sc.pl.dotplot(TRM_cd_t_specific, result_dict, groupby = j+' top clones' ,
                                  swap_axes = False ,dot_min = 0.1,standard_scale = None, title = keyname)#, save = keyname+'most_expand_DE.png' )

### Prepare the file for geneformer
A simple (lazy) alternation to the code above to save the specific TRM file to clean h5ad that can be tokenized

In [ ]:
# convert gene ID into ensembl ID - I used the reference table got from 
gene2tx = pd.read_csv('tx2gene_grch38_ens94.txt',sep = '\t',usecols = ['ensgene','symbol']).drop_duplicates(subset = 'symbol')
gene2tx.index = gene2tx['symbol']

In [ ]:
plt.rcParams.update({'font.size': 12, 'font.weight': 'bold','axes.linewidth':2})

for gtype in ['TCRab CD4','TCRab CD8ab']:
    for j in set(adata_all.obs['tissue']):
        for i in range(0,3):
            adata_temp = adatas[i][adatas[i].obs['general type']==gtype,:]
            adata_t = adata_temp[adata_temp.obs['tissue'] == j, :]
            for ct in set(adata_t.obs['general subtype']):
                keyname = ' '.join([adata_t.obs['Donor ID'][0],j,gtype,ct])
                if keyname in result_dict.keys():
                    
                    TRM_cd_t_specific = adata_t[adata_t.obs['general subtype'] == ct,:]      
                    TRM_cd_t_specific = TRM_cd_t_specific[TRM_cd_t_specific.obs[j+' top clones'].isin(['1-10','100+']),:]
                    TRM_cd_t_specific.obs['top10_or_not']= TRM_cd_t_specific.obs[j+' top clones'] =='1-10'
                    TRM_cd_t_specific.obs['top10_or_not']= TRM_cd_t_specific.obs['top10_or_not'].astype(str)
                    #if keyname == 'Donor AJG2309 L TCRab CD4 TRM':
                        #adata_for_test = TRM_cd_t_specific
                    #sc.pl.dotplot(TRM_cd_t_specific, result_dict, groupby = j+' top clones' ,
                                  #swap_axes = False ,dot_min = 0.1,standard_scale = None, title = keyname, save = keyname+'most_expand_DE.png' )
                    
                    
                    TRM_cd_t_specific = TRM_cd_t_specific[:,TRM_cd_t_specific.var_names.isin(gene2tx.index)]
                    ens_id = gene2tx['ensgene'].loc[TRM_cd_t_specific.var_names]
                    TRM_cd_t_specific.var['ensembl_id'] = ens_id
                    
                    TRM_cd_t_specific.X = TRM_cd_t_specific.layers['counts']
                    TRM_cd_t_specific.X = sp.sparse.csc_matrix(TRM_cd_t_specific.X)
                    TRM_cd_t_specific.obs =TRM_cd_t_specific.obs[['top10_or_not','activation','n_counts']]
                    
                    del TRM_cd_t_specific.uns
                    del TRM_cd_t_specific.obsm
                    del TRM_cd_t_specific.layers
                    del TRM_cd_t_specific.obsp
                    
                    TRM_cd_t_specific.obs = TRM_cd_t_specific.obs.astype(str)
                    TRM_cd_t_specific.var = TRM_cd_t_specific.var.astype(str)
                    TRM_cd_t_specific.obs['n_counts'] = TRM_cd_t_specific.obs['n_counts'].astype(float)
                    
                    TRM_cd_t_specific.write(keyname+'.h5ad')

In [ ]:
TRM_cd_t_specific

AnnData object with n_obs × n_vars = 465 × 25880
    obs: 'top10_or_not', 'activation', 'n_counts'
    var: 'ensembl_id'